In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from sklearn.preprocessing import LabelEncoder

 
# to suppress the notation 'e'    
pd.options.display.float_format = '{:.6f}'.format
from sklearn.model_selection import train_test_split
import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.tsa.api as smt
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import pip as pip
pip.main(['install', 'mlxtend'])
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import RFE

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.linear_model import SGDRegressor

# import function for ridge regression
from sklearn.linear_model import Ridge

# import function for lasso regression
from sklearn.linear_model import Lasso

# import function for elastic net regression
from sklearn.linear_model import ElasticNet

# import function to perform GridSearchCV
from sklearn.model_selection import GridSearchCV

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/flight-take-off-data-jfk-airport/M1_final.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
df_cat = data.select_dtypes(include=np.object)
df_num = data.select_dtypes(include=np.number)
print('categorical : ', df_cat.columns)
print('numerical : ', df_num.columns)

In [ ]:
plt.rcParams['figure.figsize'] = [12,12]
df_num.hist()
plt.show()

In [ ]:
#skewness and kurtosis

In [ ]:
j = []
skew = []
kurtosis = []
for i in df_num.columns[3:]:
    j.append(i)
    skew.append(data[i].skew())
    kurtosis.append(data[i].kurt())
skew_kurtosis = pd.DataFrame({'column name': j, 'skew':skew, 'kurtosis':kurtosis})
skew_kurtosis

In [ ]:
k = 1
plt.figure(figsize = (30,30))
for i in df_num.columns[3:]:
    plt.subplot(5,3,k)
    sns.distplot(data[i])
    k+=1

# EDA process


In [ ]:
plt.figure(figsize = (20,15))
sns.heatmap(data.corr(), annot = True)
plt.show()

In [ ]:
data.isnull().sum()

In [ ]:
#we find missing values in Wind

In [ ]:
data.describe().T

In [ ]:
for i in data.columns:
    print(i)
    print('---------')
    print(data[i].value_counts())
    print('+++')

In [ ]:
data[data.isnull().any(axis=1)]

In [ ]:
data['Wind'].groupby((data['DEST']=='FLL')).value_counts()

In [ ]:
data['Wind'].groupby((data['DEST']=='PWM')).value_counts()

In [ ]:
data['Wind'].value_counts()

In [ ]:
data['Wind'].replace(np.nan, 'W', inplace = True)

In [ ]:
data.isnull().sum()

In [ ]:
# null values treated

#### Outlier Treatment

In [ ]:
k = 1
plt.figure(figsize =(20,20))
for i in df_num.columns[3:]:
    if(data[i].dtypes != 'object'):
        plt.subplot(4,4,k)
        sns.boxplot(x=data[i])
        k+=1

In [ ]:
sns.boxplot(x= 'DAY_OF_WEEK', y='TAXI_OUT', data = data)

In [ ]:
Q3= data['DEP_DELAY'].quantile(0.75)
Q1= data['DEP_DELAY'].quantile(0.25)
IQR= Q3-Q1
UL= Q3 + (1.5)*IQR
LL= Q1 - (1.5)*IQR
df= data[ (data['DEP_DELAY']>=LL) & (data['DEP_DELAY']<=UL) ]

In [ ]:
df.shape

In [ ]:
k = 1
plt.figure(figsize =(20,20))
for i in df.columns[3:]:
    if(df[i].dtypes != 'object'):
        plt.subplot(5,3,k)
        sns.boxplot(x=df[i])
        k+=1

In [ ]:
data.head()

#### Model with dataset with outliers

In [ ]:
label_encoder = LabelEncoder()

data['OP_UNIQUE_CARRIER'] = label_encoder.fit_transform(data['OP_UNIQUE_CARRIER'].astype(str))
data['DEST'] = label_encoder.fit_transform(data['DEST'].astype(str))
data['Wind'] = label_encoder.fit_transform(data['Wind'].astype(str))
data['Condition'] = label_encoder.fit_transform(data['Condition'].astype(str))
data.head()

In [ ]:
data.drop("TAIL_NUM",axis=1,inplace=True)
print(data.shape)

In [ ]:
data['Dew Point'] = data['Dew Point'].astype(int)

In [ ]:
data.info()

In [ ]:
X = data.drop('TAXI_OUT',1)
Y = data['TAXI_OUT']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.1, random_state=1)
print(X_train.shape, X_test.shape ,Y_train.shape ,Y_test.shape)

In [ ]:
#OLS model

In [ ]:
Xc = sm.add_constant(X)
model_v1 = sm.OLS(Y,Xc).fit()
model_v1.summary()

In [ ]:
model_v1.resid.skew()  # Normality of Residuals

# Checking assumptions
 #  Use original data (To Build the Model)

In [ ]:
sns.distplot(model_v1.resid)

In [ ]:
#Linear Regression

In [ ]:
linear_regression = LinearRegression()
linear_regression.fit(X_train.values,Y_train.values)
rmse = (mean_squared_error(Y_test,linear_regression.predict(X_test))**0.5)
print(rmse)

In [ ]:
#Ridge Regression

In [ ]:
ridge_regression = Ridge(alpha=0.05,normalize=True)
ridge_regression.fit(X_train,Y_train)
rmse2 = (mean_squared_error(Y_test,ridge_regression.predict(X_test))**0.5)
print(rmse2)

In [ ]:
#Lasso Regression

In [ ]:
lasso_regression = Lasso(alpha=1,max_iter=1000,tol=.01)
lasso_regression.fit(X_train,Y_train)
rmse3=(mean_squared_error(Y_test,lasso_regression.predict(X_test))**0.5)
print(rmse3)

#### After outliers treatment

In [ ]:
label_encoder = LabelEncoder()

df['OP_UNIQUE_CARRIER'] = label_encoder.fit_transform(df['OP_UNIQUE_CARRIER'].astype(str))
df['DEST'] = label_encoder.fit_transform(df['DEST'].astype(str))
df['Wind'] = label_encoder.fit_transform(df['Wind'].astype(str))
df['Condition'] = label_encoder.fit_transform(df['Condition'].astype(str))
df.head()

In [ ]:
df.drop("TAIL_NUM",axis=1,inplace=True)
print(df.shape)

In [ ]:
df['Dew Point'] = df['Dew Point'].astype(int)

In [ ]:
df.info()

In [ ]:
X = df.drop('TAXI_OUT',1)
Y = df['TAXI_OUT']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.1, random_state=1)
print(X_train.shape, X_test.shape ,Y_train.shape ,Y_test.shape)

In [ ]:
#OLS model

In [ ]:
Xc = sm.add_constant(X)
model_v1 = sm.OLS(Y,Xc).fit()
model_v1.summary()

In [ ]:
model_v1.resid.skew()  # Normality of Residuals

# Checking assumptions
 #  Use original data (To Build the Model)

In [ ]:
sns.distplot(model_v1.resid)

In [ ]:
#Linear Regression

In [ ]:
linear_regression = LinearRegression()
linear_regression.fit(X_train.values,Y_train.values)
rmse_af = (mean_squared_error(Y_test,linear_regression.predict(X_test))**0.5)
print(rmse_af)

In [ ]:
#Ridge Regression

In [ ]:
ridge_regression = Ridge(alpha=0.05,normalize=True)
ridge_regression.fit(X_train,Y_train)
rmse2_af = (mean_squared_error(Y_test,ridge_regression.predict(X_test))**0.5)
print(rmse2_af)

In [ ]:
#Lasso Regression

In [ ]:
lasso_regression = Lasso(alpha=1,max_iter=1000,tol=.01)
lasso_regression.fit(X_train,Y_train)
rmse3_af=(mean_squared_error(Y_test,lasso_regression.predict(X_test))**0.5)
print(rmse3_af)

In [ ]:
models = pd.DataFrame({
'Model': ['Linear Regression','Ridge Regression',
              'Lasso Regression'],
    'rmse_before_outlier treatment': [rmse,rmse2,
              rmse3],
    'rmse_after_outlier treatment': [rmse_af,rmse2_af,
              rmse3_af]})
models

In [ ]:
Model = np.array(['Linear Regression','Ridge Regression','Lasso Regression'])
rmse_before_outlier_treatment = np.array([rmse,rmse2,rmse3])
rmse_after_outlier_treatment = np.array([rmse_af,rmse2_af,rmse3_af])
plt.plot(Model,rmse_before_outlier_treatment)
plt.plot(Model,rmse_after_outlier_treatment)
plt.legend(["rmse_before_outlier_treatment","rmse_after_outlier_treatment"])
plt.xlabel("ML Models")
plt.ylabel("RMSE")
plt.show()

In [ ]:
models.to_csv("models.csv",index = False)